参考/摘抄：https://www.cnblogs.com/z1141000271/p/9473096.html

### Tensor和Variable的合并
&emsp;&emsp;旧版本（0.1~0.3）观点认为：当前Tensor为 requires_grad = False的 Variable的特例，torch.Tensor 和 torch.autograd.Variable是同一个类，没有本质的区别.
实际上，已经没有纯粹的Tensor，所有的Tensor对象都应支持自动求导。
> 查看pytorch的版本类型
```
print(torch.__version__)
```

### 查看Tensor的类型
&emsp;&emsp; 使用「`isinstance()`」或者「`x.type()`」,但是使用python内置函数type(x)不能查看tensor变量的实际类型；

In [1]:
from __future__ import print_function
import torch as t

In [3]:
x = t.DoubleTensor([1, 1, 1])
print(x.type()) # was torch.DoubleTensor

print(type(x))
print('the variable x is a torch.FloatTensor: ', isinstance(x, t.FloatTensor))

torch.DoubleTensor
<class 'torch.Tensor'>
the variable x is a torch.FloatTensor:  False


### Tensor的requires_grad属性
&emsp;&emsp;已经没有纯粹的tensor变量，任何一个tensor对象/变量都支持自动求导,默认求导属性为False，即 requires_grad = False;
#### requires_grad属性使用
- 可将 requires_grad属性作为参数，构造tensor变量

In [6]:
x = t.ones(1)
x.requires_grad  # 默认为False
y = t.ones(1)
z = x + y
z.requires_grad  # 根据传导性，z的 requires_grad属性 为False
# z.backward()     # 因为所有变量都不需要grad，因此会 Error
print(z.grad_fn)

None


In [16]:
# 将 requires_grad属性作为参数，构造 tensor变量
w = t.ones(1, requires_grad = True)
total = x + w
print('the sum variable requires_grad and grad_fn is : \n', total.requires_grad, total.grad_fn)

# 此时可以 进行 求导，调用 backward函数
total.backward()


the sum variable requires_grad and grad_fn is : 
 True <ThAddBackward object at 0x7f1bc7a9a5c0>


In [21]:
print('the derivative is : ',w.grad,x.grad)

# 因为 x,y,z 都是不需要梯度的（ requires_grad = False）,因此 x.grad,y.grad,z.grad均为None未进行计算
print(w.grad,x.grad,y.grad,z.grad)

the derivative is :  tensor([1.]) None
tensor([1.]) None None None


####  **`.data`**不要随意使用
&emsp;&emsp;早期版本中，使用.data获得Variable变量中的Tensor。后期，Tensor和Variable进行了合并，因此「.data」返回一个新的requires_grad = False的Tensor！！！
**注意：**新的Tensor同之前的Tensor变量共享内存，所以不安全

In [24]:
print('before time x.requires_grad = ', x.requires_grad)
y = x.data # x 需要进行 autograd
print('after time x.requires_grad = ', x.requires_grad)
# y和x 共享内容，但是这里并不需要grad了
# 所以会导致本来需要进行梯度的x也没有梯度可以计算了，从而x不会得到更新
z = x.detach()
print('after time x.requires_grad = ', x.requires_grad)

before time x.requires_grad =  True
after time x.requires_grad =  True
after time x.requires_grad =  True


#### scale 的支持
&emsp;&emsp;早期版本中，indexing 一个一维Tensor，返回一个number类型（标量）；但是indexing 一个Variable 确实返回一个size = （1，） 的 **vector**；再比如，tensor.sum()返回一个number,但variable.sum()返回的是一个size = （1，）的vector
** 好处** 
1. 通过引入`scalar`，可以将返回值的类型进行统一；
2. 取得一个tensor的值（返回number），需要使用成员函数「.item()」
3. 创建 scalar 的话， 需要使用 「torch.tensor(number)」
4. torch.tensor（list）也可以创建 tensor

**注意：** 
- tensor.sum(): tensor表示一个Tensor类型的变量
- variable.sum(): variable表示一个Variable类型的变量
- scalar 打印出来是没有 **[ ]**的， tensor 打印出来会用 `[]`包上
- scalar: 0-dimension 的 tensor

In [35]:
import torch
torch.tensor(3.141592627) # 使用 torch.tensor来创建scalar
torch.tensor(3.1415).size()  # size为 0 , was torch.Size([])
torch.tensor([3]).size()    # compare to a vecotor of size 1
#torch.tensor([2])          # 如果为 tensor,打印出来用 `[]`包上
vector = torch.arange(2, 6) # this is a vector
vector # 类似为： vector = torch.tensor([2, 3, 4, 5])

# 获取 tensor中的值（vector） 
vector[3].item()   # 需要使用成员函数 「.item()」获取里面的值

# 而这种 reduction 操作，返回的是一个 scalar (0-dimension 的 tensor)
mySum = torch.tensor([2, 4, 6]).sum()
mySum

tensor(12)

------------------------------------------------------
// TODO
#### 累加 loss
&emsp;&emsp;早期版本中，累加loss一般使用 「totalLoss += loss.data[0]」,后期版本累加loss，采用 「loss.item()」
##### 为啥采用 「.data[0]」
**原因：**loss是一个Variable， 

#### 弃用 volatile
现在这个flag 已经启用了，被替换成为 torch.no_grad(), torch.set_grad_enable(grad_mode)等函数

In [37]:
x = torch.zeros(1, requires_grad = True)
with torch.no_grad():
    y = x ** 2
    print('has enter the with function')
y.requires_grad

has enter the with function


False

In [39]:
is_train = False
with torch.set_grad_enabled(is_train):
    y = x ** 2
    print('has enter the with function')
y.requires_grad

has enter the with function


False

In [40]:
torch.set_grad_enabled(True) # this can also used as a funciton
y = x ** 2
y.requires_grad

True

### dtypes, devices 以及 numpy-style 的构造函数
dtype 是data types，对应关系如下：
![image](https://github.com/stonels0/pytorch-book/tree/master/chapter2-%E5%BF%AB%E9%80%9F%E5%85%A5%E9%97%A8/imgs/dtype.jpeg) 
通过**.dtype**可以得到，其他就是以前写 device.type 都是用 .cup() 或者 .cudn() ，现在独立成为一个函数

In [60]:
device = torch.device("cuda:0")
x = torch.randn(3, 3, dtype=torch.float64, device=device)

### 创建新的 Tensor方法
1. 可以指定dtype 和 device 创建;
2. 用 torch.tensor 创建 Tensor；
3. 用 torch. * like 以及 torch.new_* 进行创建;
&emsp;&emsp; like: 用于创建 「shape相同，数据类型相同」

&emsp;&emsp; new_：用于创建属性相同，shape不想要一致


In [59]:
# 指定 dtype 和 device进行创建
device = torch.device("cuda:0")
x = torch.randn(3, 3, dtype = torch.float64,device=device) # 当前使用device参数报错，后期研究 TODO
# x.requires_grad # was False
x

tensor([[ 0.9927, -1.5810, -0.1452],
        [ 0.1451,  0.4920,  1.3241],
        [ 0.8099, -0.6106,  0.3512]], device='cuda:0', dtype=torch.float64)

In [62]:
# 使用 torch.tensor创建Tesor，类似于 numpy.array()
# 从列表中list的数据来创建
cuda = torch.device("cuda")
torch.tensor([[1], [2], [3]], dtype = torch.half, device=cuda)
# 创建 scalar 
torch.tensor(1)  # scalar 打印输出 不包含【】

tensor(1)

In [63]:
# torch.*like : 创建「shape、数据类型」相同
x = torch.randn(3, dtype = torch.float64)
torch.zeros_like(x)
torch.zeros_like(x, dtype = torch.int)

tensor([0, 0, 0], dtype=torch.int32)

In [67]:
y = x.new_ones(2) # 属性相同,均为 torch.float64类型的
print(y)

tensor([1., 1.], dtype=torch.float64)


#### 书写 device-agnostic
含义：不需要显示指定是 GPU，CPU之类，直接利用 「.to()」来执行

In [70]:
# at beginning of the script
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# then whenever you get a new Tensor or Module
# this won't copy if they are already on the desired device
input = data.to(device)
model = MyModule(...).to(device)

NameError: name 'data' is not defined

#### 迁移代码对比
- 以前写法:
```
 model = MyRNN()
  if use_cuda:
      model = model.cuda()

  # train
  total_loss = 0
  for input, target in train_loader:
      input, target = Variable(input), Variable(target)
      hidden = Variable(torch.zeros(*h_shape))  # init hidden
      if use_cuda:
          input, target, hidden = input.cuda(), target.cuda(), hidden.cuda()
      ...  # get loss and optimize
      total_loss += loss.data[0]

  # evaluate
  for input, target in test_loader:
      input = Variable(input, volatile=True)
      if use_cuda:
          ...
      ...
```

- 迁移后的写法：
```
 # torch.device object used throughout this script
  device = torch.device("cuda" if use_cuda else "cpu")

  model = MyRNN().to(device)

  # train
  total_loss = 0
  for input, target in train_loader:
      input, target = input.to(device), target.to(device)
      hidden = input.new_zeros(*h_shape)  # has the same device & dtype as `input`
      ...  # get loss and optimize
      total_loss += loss.item()           # get Python number from 1-element Tensor

  # evaluate
  with torch.no_grad():                   # operations inside don't track history
      for input, target in test_loader:
          ...
```